In [ ]:
import json
from multiprocessing import Pool
import os
from pathlib import Path
import sqlite3
from tqdm import tqdm

import pandas as pd
from spacy.lang.en import English

from bbsearch.utils import get_tag_and_sentences, define_nlp, update_covid19_tag, insert_into_sentences

## Load data: `metadata.csv` and find `json` files

In [ ]:
DATASET_VERSION = 'v7'

p = Path('/raid/covid_data/data') / DATASET_VERSION

n_json_files = len(list(p.glob('**/*json')))

print(f'Found {n_json_files:,d} JSON files for the CORD-19 (version {DATASET_VERSION}).')

In [ ]:
df = pd.read_csv(p / 'CORD-19-research-challenge' / 'metadata.csv')

print(f'Found {len(df):,d} article entries for the CORD-19 (version {DATASET_VERSION})')

## Create associative table `article_id_2_sha`

In [ ]:
df2 = pd.DataFrame(data=df[['cord_uid', 'sha']])
df2 = df2.set_index(['cord_uid']).apply(lambda x: x.str.split('; ').explode()).reset_index()
df2.rename(columns={'cord_uid':'article_id', 'sha':'sha'}, inplace=True)

In [ ]:
with sqlite3.connect(f'cord19_{DATASET_VERSION}.db') as db:
    conn = db.cursor()

    db.execute(
        """CREATE TABLE IF NOT EXISTS article_id_2_sha
        (
            article_id TEXT,
            sha TEXT
        );
        """
        )

    df2.to_sql(name='article_id_2_sha', con=db, index=False, if_exists='append')
    
    display(pd.read_sql('SELECT * FROM article_id_2_sha LIMIT 5;', db))

## Create `articles` table

In [ ]:
df.rename(columns={
    'cord_uid':'article_id', 
    'sha':'sha',
    'source_x':'publisher', 
    'title':'title', 
    'doi':'doi', 
    'pmcid':'pmc_id', 
    'pubmed_id':'pm_id', 
    'license':'licence',
    'abstract':'abstract', 
    'publish_time':'date', 
    'authors':'authors', 
    'journal':'journal',
    'Microsoft Academic Paper ID':'microsoft_id', 
    'WHO #Covidence':'covidence_id', 
    'has_pdf_parse':'has_pdf_parse',
    'has_pmc_xml_parse':'has_pmc_xml_parse', 
    'full_text_file':'fulltext_directory', 
    'url':'url'
}, inplace=True)

In [ ]:
df.head(n=2)

In [ ]:
for c_ in df.columns:
    print(f'column {repr(c_):>20s} has {df[c_].isna().sum():>15,d} NULL')

### Drop column `sha`, for which we have the associative table

In [ ]:
df.drop('sha', axis=1, inplace=True)

### Some `article_id` appear twice, so drop duplicates

In [ ]:
df.drop_duplicates('article_id', keep='first', inplace=True)

### Create table

In [ ]:
with sqlite3.connect(f'cord19_{DATASET_VERSION}.db') as db:
    db.execute(
        """CREATE TABLE IF NOT EXISTS articles
        (
            article_id TEXT PRIMARY KEY, 
            publisher TEXT, 
            title TEXT, 
            doi TEXT, 
            pmc_id TEXT, 
            pm_id INTEGER, 
            licence TEXT,
            abstract TEXT, 
            date DATETIME, 
            authors TEXT, 
            journal TEXT,
            microsoft_id INTEGER, 
            covidence_id TEXT, 
            has_pdf_parse BOOLEAN,
            has_pmc_xml_parse BOOLEAN, 
            has_covid19_tag BOOLEAN DEFAULT False,
            fulltext_directory TEXT, 
            url TEXT
        );
        """
        )

    df.to_sql(name='articles', con=db, index=False, if_exists='append')

    display(pd.read_sql('SELECT * FROM articles LIMIT 3', db))

## Create `sentences` table

In [ ]:
with sqlite3.connect(f'cord19_{DATASET_VERSION}.db') as db:
    db.execute("""DROP TABLE IF EXISTS sentences""")
    db.execute(
            #"""CREATE TABLE IF NOT EXISTS sentences
            """CREATE TABLE sentences
            (
                sentence_id INTEGER PRIMARY KEY AUTOINCREMENT,
                sha TEXT,
                section_name TEXT,
                text TEXT,
                FOREIGN KEY(sha) REFERENCES article_id_2_sha(sha)
            );
            """
            )

In [ ]:
with sqlite3.connect(f'cord19_{DATASET_VERSION}.db') as db:
    display(pd.read_sql("SELECT article_id, COUNT() c FROM article_id_2_sha GROUP BY article_id ORDER BY c DESC LIMIT 5", db))

In [ ]:
nlp = define_nlp()
all_tags_pos = 0
all_tags = 0
nb_sentences = 0
nb_sentences_tagged = 0 
 
with sqlite3.connect(f'cord19_{DATASET_VERSION}.db') as db:

    cur = db.cursor()
    for (article_id, ) in tqdm(cur.execute('SELECT article_id FROM articles')):
        tag, sentences = get_tag_and_sentences(db, nlp, p, article_id)
        all_tags_pos += tag
        all_tags += 1
        nb_sentences += len(sentences) 
        if tag:
            nb_sentences_tagged += len(sentences)
        update_covid19_tag(db, article_id, tag)
        insert_into_sentences(db, sentences)
        if all_tags == 10:
            break

    db.commit()

In [ ]:
pd.read_sql("""SELECT COUNT() FROM sentences""", db)
#pd.read_sql("""SELECT * FROM articles WHERE has_covid19_tag=True LIMIT 10""", db)

In [ ]:
print('Number of articles: ', all_tags)
print('Number of sentences: ', nb_sentences)
print('Number of articles tagged covid19: ', all_tags_pos)
print('Number of sentences tagged covid19: ', nb_sentences_tagged)

In [ ]:
pd.read_sql("""SELECT * FROM sentences""", db)

In [ ]:
pd.read_sql("""SELECT COUNT() FROM articles WHERE has_covid19_tag=1""", db)